In [46]:
import tensorflow as tf 
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [47]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")



In [48]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [49]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

In [50]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [51]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [52]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  #print(gradients)

  train_loss(loss)
  train_accuracy(labels, predictions)
  return gradients

In [53]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [54]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    gradients = train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

KeyboardInterrupt: 

In [58]:
gradients = train_step(images, labels)
gradients 

[<tf.Tensor: shape=(3, 3, 1, 32), dtype=float32, numpy=
 array([[[[-7.41174771e-03, -1.52880780e-03, -1.16648013e-02,
            8.31065886e-03, -2.31368672e-02, -3.64221074e-03,
            3.13796481e-05, -3.06823896e-03,  2.02698447e-02,
           -1.30772078e-02, -1.48162628e-02, -2.39507239e-02,
           -7.12619489e-03,  1.55112594e-02,  4.26999200e-03,
           -5.67619689e-03, -4.22231155e-03, -3.38581055e-02,
            5.93247637e-03,  1.04281697e-02, -2.22555618e-03,
           -2.17144424e-03, -1.03486644e-03,  6.41715713e-03,
           -1.35254972e-02, -8.48580711e-03, -8.83912109e-03,
           -6.61753630e-03, -1.63528714e-02,  9.13019106e-03,
           -5.50198415e-03,  9.89014842e-03]],
 
         [[-9.70854145e-03, -5.74327260e-03, -1.32905375e-02,
            2.20118975e-03, -1.43601801e-02, -2.29229331e-02,
           -6.00388739e-06, -2.05392647e-03,  2.10146401e-02,
           -1.62162259e-02, -2.27993093e-02, -1.26660401e-02,
           -7.95980170e-03,